# Creating and Deploying an R web service with Azure Machine Learning Services
This notebook shows the steps required for writing and deploying an Azure Machine Learning Service which runs R code in either ACI (Azure Container Instances) or AKS (Azure Kubernetes Service).

# Get workspace
Load existing workspace from the config file info. If you have not prepared the config file yet, use the `configuration.ipynb` notebook to create it.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Register the model
Register an existing trained model, add description and tags.

In [ ]:
from azureml.core.model import Model
model = Model.register(model_path = "model.RData",
                       model_name = "model.RData",
                       tags = {'area': "samples", 'type': "regression"},
                       description = "A simple linear regression model to show the usage of R in Azure Machine Learning Services.",
                       workspace = ws)

print(model.name, model.description, model.version)

# Create an image
Create an image using the registered model the script that will load and run the model.

In [ ]:
%%writefile score.py
from azureml.core.model import Model
import rpy2.rinterface
import rpy2.robjects as robjects

def init():
    # init rpy2
    rpy2.rinterface.initr()
    # load model
    model_path = Model.get_model_path('model.RData')
    robjects.r("load('{model_path}')".format(model_path=model_path))
    # run init() function in R (if exists)
    robjects.r("if (exists('init', mode='function')) { init() }")

def run(input_json_string):
    try:
        result_vector = robjects.r(
                "run('{input_json_string}')".format(input_json_string=input_json_string)
            )
        if len(result_vector) > 0:
            return result_vector[0]
        else:
            return ""
    except Exception as e:
        error = str(e)
        return error

Define the dependencies required by the scoring script to run.

- rpy2 is a package to run R from Python
- mro-base is Microsoft's R distribution
- r-jsonlite and r-essentials are two conda packages which add the jsonlite package and a few other common packages to our R

If you need more packages, you can specify them here - which is the most convenient method. Another option is to install additional packages within the R or Python code (in cases internet connectivity is available) or provide a custom docker image.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

conda_dependencies = CondaDependencies.create(conda_packages=['rpy2', 'mro-base','r-jsonlite','r-essentials'])

with open("conda_dependencies.yml","w") as f:
    f.write(conda_dependencies.serialize_to_string())

Create the image (this might take a while).

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "conda_dependencies.yml",
                                                  description = "Sample image to show the usage of R with Azure Machine Learning Services.",
                                                  tags = {'area': "samples", 'type': "regression"},
                                                 )

image = ContainerImage.create(name = "r-on-amls-sample",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

# Deployment
## Option 1: Deploy to ACI

Note: the example shown here for ACI does not use SSL! See the description [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-secure-web-service) to learn how this can be enabled. Getting SSL to work is easier for AKS clusters (see below).

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

myaci_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 2,
    tags = {'area': "samples", 'type': "regression"},
    description = 'Sample image to show the usage of R with Azure Machine Learning Services.'
)

service_name = "r-on-aml-service-aci"
runtime = "python" 
driver_file = "score.py"
my_conda_file = "conda_dependencies.yml"

aci_service = Webservice.deploy_from_model(
  workspace=ws, 
  name=service_name,
  deployment_config = myaci_config,
  models = [model],
  image_config = myimage_config
)
aci_service.wait_for_deployment(show_output=True)

print("Scoring URI: {}".format(aci_service.scoring_uri))
print("Swagger URI: {}".format(aci_service.swagger_uri))

## Option 2: AKS (Kubernetes)


In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

aks_name = 'akscluster'

### If not setup yet: Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

The example shows both SSL and non-SSL deployment. Note that the cluster has to be created in the right manner if SSL shall be used. Adding SSL afterwards is not easy - if possible at all.

See the included `Create-SelfSignedCertificate.ps1` file for creating self-signed certificates (Windows Powershell/Bash, will be different for Linux/Mac).

> Note:
>
> Self-signed certificates should not be used in production systems. In production systems, proper (and probably paid)
> keys and certificates should be used.
>
> Applications that consume the published webservices may (by design) not behave as intended if self-signed certificates are used.

In [ ]:
# non-SSL
prov_config = AksCompute.provisioning_configuration()
# SSL-enabled
#prov_config = AksCompute.provisioning_configuration(
#    ssl_cert_pem_file="<file with the certificate, eg. certificate_for_amls.pem>",
#    ssl_key_pem_file="<file with the key, eg. key_for_amls.pem>",
#    ssl_cname="<add the cname from the certificate here>"
#)

aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [ ]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

**Further config adjustments NOT TO BE SKIPPED** if SSL/TLS is used

Now adjust the clusters DNS settings as described here:
https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-secure-web-service#update-your-dns

IMPORTANT: the DNS setting in Azure Portal and the cname in the certificate need to match! 

### If setup already: Attach existing AKS cluster

If you have existing AKS cluster in your Azure subscription, you can attach it to the Workspace.

In [ ]:
'''
# Use the default configuration (can also provide parameters to customize)
resource_id = '/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourcegroups/raymondsdk0604/providers/Microsoft.ContainerService/managedClusters/my-aks-0605d37425356b7d01'

create_name='my-existing-aks' 
# Create the cluster
attach_config = AksCompute.attach_configuration(resource_id=resource_id)
aks_target = ComputeTarget.attach(workspace=ws, name=create_name, attach_configuration=attach_config)
# Wait for the operation to complete
aks_target.wait_for_completion(True)
'''

### Deploy web service to AKS

In [ ]:
aks_target = ComputeTarget(workspace = ws, name = aks_name)
aks_config = AksWebservice.deploy_configuration()

In [ ]:
%%time
aks_service_name ='r-on-aml-service'
aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

In [ ]:
print("Scoring URI: {}".format(aks_service.scoring_uri))
print("Swagger URI: {}".format(aks_service.swagger_uri))
print("Keys: {}".format(aks_service.get_keys()))

Note: By default, the deployed service uses HTTP only. To secure your service see and follow the steps described [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-secure-web-service).

# Test the web service
We test the web sevice by passing data.

Note: the test may fail if you test against a self-signed certificate **which is ok**. See the `consume-webservice.ipynb` notebook for an example how you can circumvent the certificate check.

In [ ]:
%%time
import json

test_sample = json.dumps({'x': 987.654})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aks_service.run(input_data = test_sample)

print(prediction)

# Clean up
Delete the service, image and model.

In [ ]:
%%time
aks_service.delete()
image.delete()
model.delete()